## Initialization Block

In [1]:
import csv
from collections import defaultdict
candidates = []
freq = defaultdict(int)
count = 0
with open("unigram_freq.csv", newline="") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        if count >= 5000:
            break
        if len(row[0]) == 5 and row[0].isalpha():
            candidates.append(row[0])
            freq[row[0]] = int(row[1])
            count += 1
stat = defaultdict(set)
for candidate in candidates[:5000]:
    for letter in "abcdefghijklmnopqrstuvwxyz":
        if letter in candidate:
            stat[letter].add(candidate)
        for pos in range(5):
            if candidate[pos] == letter:
                stat["".join([letter, str(pos)])].add(candidate)
stat[""] = set(candidates)
def minusOp(total, negs, stat):
    for neg in negs:
        total -= stat[neg]
    return total
def knownOp(conds, stat):
    if not conds:
        return set(stat[""])
    total = set(stat[conds[0]])
    for cond in conds[1:]:
        total &= stat[cond]
    return total
    
def apply_cond(stat, knowns, negs):
    cand = knownOp(knowns, stat)
    cand = minusOp(cand, negs, stat)
    return cand


## Solver Block

In [13]:
def solver():
    print("Given your prompts in this manner: color [Black Yellow Green Black Green] = _*^_^")
    print("Type 'exit' in guess input to exit.")
    conds = set()
    negs = set()
    while True:
        while True:
            guess = input("What did you put(guess): ")
            if guess == ("exit" or result == "*****") or (len(guess) == 5):
                break
            else:
                print("Invalid guess, please try again")
        while True:
            result = input("What is the returned(result): ")
            valid = len(result) == 5
            for r in result:
                if r not in "_^*":
                    valid = False
            if valid:
                break
            else:
                print("Not a valid result, please try again")
        if guess == "exit" or result == "*****":
            break
        for i, (g, elem) in enumerate(zip(guess, result)):
            if elem == "_":
                if g not in conds:
                    negs.add(g)
            elif elem == "*":
                negs.add("".join([g,str(i)]))
                conds.add(g)
            else:
                conds.add("".join([g,str(i)]))
                conds.add(g)
        maybe = apply_cond(stat, list(conds), list(negs))
        maybe = sorted(list(maybe), key=lambda k:-freq[k])
        print("Your best", "shot is" if len(maybe) <= 1 else "shots are", maybe if len(maybe) > 0 else None)
        if len(maybe) == 0:
            print("No vaid solution found, check inputs closely.")
            return
solver()

Given your prompts in this manner: color [Black Yellow Green Black Green] = _*^_^
Type 'exit' in guess input to exit.
What did you put(guess): start
What is the returned(result): __^__
Your best shots are ['email', 'black', 'place', 'image', 'again', 'beach', 'leave', 'peace', 'claim', 'heavy', 'chain', 'miami', 'coach', 'plain', 'idaho', 'piano', 'blank', 'plane', 'plaza', 'blade', 'omaha', 'flame', 'blame', 'ghana', 'diane', 'diana', 'whale', 'blake', 'quake', 'icann', 'chang', 'peach', 'zhang', 'avail', 'playa', 'clamp', 'awake', 'gmail', 'blanc', 'champ', 'kuala', 'gland', 'chalk', 'blaze', 'huang', 'alamo', 'plano', 'weave', 'duane', 'meade', 'alain', 'plaid', 'avaya', 'bland', 'keane', 'leach', 'khaki', 'glaze', 'anand', 'jiang', 'quail', 'plank', 'glade', 'dhabi', 'qmail', 'healy', 'knapp', 'llama', 'dhaka', 'ocala', 'koala', 'flank', 'aland', 'liang', 'flake', 'leahy', 'amaya', 'doanh', 'hwang', 'joann', 'amana', 'amaze', 'plame', 'gmake', 'cpage', 'ocaml', 'whack', 'leafy', 'n

## Play Block

In [ ]:
import random
def play(ans = None):
    print("Type 'I give up' to end game.")
    print("Type 'Show hints' to reveal hints")
    if not ans:
        ans = random.choice(candidates)
    conds = set()
    negs = set()
    guess = ""
    magic_words = ["I give up", "Show hints"]
    while guess != ans:
        while True:
            guess = input("Your guess: ")
            if guess in stat[""] or guess in magic_words :
                break
            else:
                print("Not valid input, must be an English word or magic words")
        if guess == "I give up":
            break
        if guess == "Show hints":
            print(maybe)
            continue
        guess.lower()
        maybe, negs, conds, prompt = step(stat, guess, ans, negs, conds)
        print(prompt)
    if guess == ans:
        print("You got it! It is " + ans + ".")
    else:
        print("Try next time! The answer is " + ans + ".")
    
def step(stat, guess, ans, negs, conds):
    prompt = ""
    for i in range(5):
        if guess[i] == ans[i]:
            prompt += guess[i].capitalize()
            conds.add("".join([guess[i],str(i)]))
            conds.add(g)
        elif guess[i] in ans:
            prompt += guess[i]
            conds.add(guess[i])
            negs.add("".join([guess[i],str(i)]))
        else:
            negs.add(guess[i])
            prompt += "_"
#         print(conds, negs)
    maybe = apply_cond(stat, list(conds), list(negs))
    maybe = sorted(list(maybe), key=lambda k:-freq[k])[:50]
    return maybe, negs, conds, prompt

play()

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
def experiment(policy = lambda maybe, knows: random.choice(maybe)):
    num_exp = 10000
    steps_token = []
    for i in tqdm(range(num_exp)):
        guess = "ideal"
        ans = random.choice(candidates)
#         print(guess, ans)
        conds = set()
        negs = set()
        steps = 1
        while guess != ans:
            maybe, negs, conds, prompt = step(stat, guess, ans, negs, conds)
#             print(prompt, maybe, ans)
            guess = policy(maybe, (negs, conds))
#             print(guess)
            steps += 1
        steps_token.append(steps)
#         print(len(stat[""]))
    print(sum(steps_token)/num_exp)
    plt.hist(steps_token)
    plt.show()
experiment()